# Ugesco : enrich and desambiguate the Samnang-Hans' JSON

In [1]:
from ugesco import *

In [2]:
file = "data/jsondata_ugesco.json"
data = json_to_df(file)

In [3]:
# exclude image classifications and temporal values for the moment
data = data[data.columns.drop(list(data.filter(regex='imageclassification|temporal')))]

In [4]:
# reshape the dataframe verticaly
data = data.set_index('beeldid')
data.columns = data.columns.str.split('_', expand=True)
data = data.stack().reset_index(level=1, drop=True).reset_index()
data.columns = ['beeldid', 'spatial_value', 'spatial_key', 'to_drop']
data.drop('to_drop', 1, inplace=True)

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2862: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2862: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
#join/merge with phototheque_pallas to get the locations. 
#The csv is gzipped to get around the 100MB file weigth limitation of Github
phototeque = pd.read_csv("data/phototheque_pallas.csv.gz", compression="gzip", encoding="utf8", dtype={'beeldid': str})

data = pd.merge(data, phototeque, how='left', on=['beeldid'], suffixes=['', '_x'])

In [7]:
# Pictures that contains more than a loc in the thesaurus descriptors

more_than_one_loc = data[data['loc_qid'].str.contains(",", na = False)]

In [ ]:
#test Rosette API. Max 10 000 calls a month and 1000 a day
        
#data['rosette'] = data['LEGEND'].apply(rosette)

In [8]:
# Some descriptive stats on columns
data.describe(include="all")

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name
count,1166,1166,1166,1166,1166,1166,1166.000000,782,951,951,620,626
unique,655,409,8,655,655,296,NaN,152,8,8,63,65
top,00035613,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Le corps du baron de Borchgrave de l'Ambassade...,"Libération de Verviers par les alliés, 9/1944....",NaN,vie pratique--bruxelles (belgique),Q31,Belgium,Q239,Brussels
freq,9,299,715,9,9,56,NaN,68,904,904,300,300
mean,NaN,NaN,NaN,NaN,NaN,NaN,16531.884220,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,41527.337298,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,521.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,1756.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,3144.000000,NaN,NaN,NaN,NaN,NaN


In [9]:
# More data profiling
import pandas_profiling
pandas_profiling.ProfileReport(data)

Number of variables,12
Number of observations,1166
Total Missing (%),13.6%
Total size in memory,118.4 KiB
Average record size in memory,104.0 B
Numeric,1
Categorical,11
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [10]:
#data.to_csv("C:/Users/ettor/Desktop/ugesco_file_temp_simplified.csv", encoding="utf-8")

In [12]:
# Merge with NER_classes (spatial_keys matched with wikidata)
ner_classes = pd.read_csv("data/ner_classes.csv")

data = pd.merge(data, ner_classes, how='left', left_on = ['spatial_key'], right_on = ['ner_class'])

In [15]:
# rename the new columns merged
data.rename(columns={'wiki_qid': 'ner_class_qid', 'wiki_class':'ner_class_name'}, inplace=True)

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class,ner_class_qid,ner_class_name
0,00009500,Brussel,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Brussel. L'église St.e. Marie. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity
1,00009500,L' église St . e. Marie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Brussel. L'église St.e. Marie. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_BUILDING,Q41176,building
2,00009520,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Grand' Place.,Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity
3,00009520,Grand ' Place,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Grand' Place.,Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_STREET,Q83620,thoroughfare
4,00009537,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. La Château de Tervuren. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity


In [23]:
# remove articles in spatial_value
data['spatial_value'] = data['spatial_value'].str.replace(pat=r"^\s?(le|la|l\s+'|l'|les)\s+", repl="", n=1, case=False)

In [24]:
data.head()

,beeldid,spatial_value,spatial_key,lien_phototheque,LEGEND,BPALLASF,trfwnumm,keyword,country_qid,country_name,loc_qid,loc_name,ner_class,ner_class_qid,ner_class_name
0,00009500,Brussel,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Brussel. L'église St.e. Marie. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity
1,00009500,église St . e. Marie,LOC_BUILDING,http://pallas.cegesoma.be/pls/opac/opac.search...,Brussel. L'église St.e. Marie. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_BUILDING,Q41176,building
2,00009520,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Grand' Place.,Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity
3,00009520,Grand ' Place,LOC_STREET,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. Grand' Place.,Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_STREET,Q83620,thoroughfare
4,00009537,Bruxelles,LOC_PLACE,http://pallas.cegesoma.be/pls/opac/opac.search...,Bruxelles. La Château de Tervuren. [Sipho],Vues de Bruxelles : 1942-1944. - 124 photos,214.0,bruxelles (belgique)--photographies,Q31,Belgium,Q239,Brussels,LOC_PLACE,Q56061,administrative territorial entity


In [ ]:
#apply the algorithm
for index, rows in data.iterrows():
    dic = {}
    dic['index'] = index
    dic['beeldid'] = rows.beeldid
    if ("," not in str(rows.loc_wikiname) 
        and str(rows.loc_wikiname) != "nan"
        and isQidACity(rows.loc_qid)): 
        dic['city_thes_name'] = rows.loc_wikiname
        dic['city_qid_name'] = rows.loc_qid
        #dic['country_name'] = rows.country_name
        #dic['country_qid'] = rows.country_qid
    #elif (rows.spatial_key_0 == "LOC_PLACE" 
          #and isStringACity(rows.spatial_value_0)):
        #dic['city'] = rows.spatial_value_0
    print(dic)

          
        